# Data Acquisition Process for WiFi Signal Strength Analysis

## Overview

This notebook implements the data acquisition process for collecting and processing WiFi signal strength data from access points (APs) in a building. The process involves:

1. **Environment Setup**: Import libraries and configure parameters
2. **Data Loading**: Read data from JSON/CSV files or InfluxDB
3. **Data Processing**: Filter 2.4GHz band, clean data, and process BSSID information
4. **AP Integration**: Merge AP location data from coordinate files
5. **Data Export**: Save processed data to CSV format

## Data Sources

- **InfluxDB**: Database option (configurable but not currently used)
- **JSON File**: Primary data source (aruba_07_15.json) with WiFi measurements
- **AP Coordinate Files**: CSV files (3f.csv, 2f.csv) with AP location data

## 1. Environment Setup

Import necessary libraries and load environment variables.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import json
import os
import yaml
from datetime import datetime
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# InfluxDB imports (optional - install with: pip install influxdb-client)
try:
    from influxdb_client import InfluxDBClient, Point, WriteOptions
    from influxdb_client.client.query_api import QueryOptions
    from influxdb_client.client.warnings import MissingPivotFunction
    warnings.simplefilter("ignore", MissingPivotFunction)
    INFLUXDB_AVAILABLE = True
except ImportError:
    print("InfluxDB client not available. Install with: pip install influxdb-client")
    INFLUXDB_AVAILABLE = False

# Load environment variables
load_dotenv()

# Configuration parameters
USE_INFLUXDB = True  # Set to True to use InfluxDB, False to use file
INFLUXDB_CREDENTIALS_PATH = "credentials.yml"  # Path to InfluxDB credentials

# Available data files found in the system
DATA_SOURCE_CSV = "/home/sionna/Documents/ghulam/rssi_d1_2f.csv"  # RSSI data for 2F
DATA_SOURCE_JSON = "/home/sionna/Documents/GitHub/nvidia-sionna/AP BSS Table.json"  # AP BSS table
ALTERNATIVE_DATA_FILES = {
    "full_rssi_d1": "/home/sionna/Documents/ghulam/full_rssi_d1.csv",
    "nvidia_rssi_2f": "/home/sionna/Documents/GitHub/nvidia-sionna/full_rssi_d1_2f.csv",
    "mapped_rssi": "/home/sionna/Documents/GitHub/nvidia-sionna/mapped_rssi_d1_2f.csv"
}

# Since we don't have coordinate files, we'll create sample ones or extract from AP names
AP_COORD_FILES = {
    "2f": "2f.csv",  # Will be created if not found
    "3f": "3f.csv"   # Will be created if not found
}
OUTPUT_FILE = "ap_data.csv"

print("Environment setup complete!")
print(f"InfluxDB available: {INFLUXDB_AVAILABLE}")
print(f"Use InfluxDB: {USE_INFLUXDB}")
print(f"InfluxDB credentials: {os.path.exists(INFLUXDB_CREDENTIALS_PATH) if USE_INFLUXDB else 'N/A'}")
print(f"Primary RSSI data source: {DATA_SOURCE_CSV}")
print(f"AP BSS table: {DATA_SOURCE_JSON}")
print(f"Alternative data files available: {len(ALTERNATIVE_DATA_FILES)}")
print(f"Output file: {OUTPUT_FILE}")

# Check which files actually exist
print(f"\nFile availability check:")
if USE_INFLUXDB and INFLUXDB_AVAILABLE:
    print(f"  InfluxDB credentials exist: {os.path.exists(INFLUXDB_CREDENTIALS_PATH)}")
print(f"  RSSI CSV exists: {os.path.exists(DATA_SOURCE_CSV)}")
print(f"  AP BSS JSON exists: {os.path.exists(DATA_SOURCE_JSON)}")
for name, path in ALTERNATIVE_DATA_FILES.items():
    print(f"  {name}: {os.path.exists(path)}")

## 2. Load Data from File or InfluxDB

Load WiFi signal strength data from the configured source (JSON file or InfluxDB).

In [ ]:
def load_data_from_file(file_path):
    """Load data from JSON or CSV file"""
    try:
        if file_path.endswith('.json'):
            with open(file_path, 'r') as f:
                data = json.load(f)
            print(f"Successfully loaded JSON data from {file_path}")
            return data
        elif file_path.endswith('.csv'):
            # For large CSV files, read in chunks to avoid memory issues
            try:
                data = pd.read_csv(file_path)
                print(f"Successfully loaded CSV data from {file_path}")
                print(f"Shape: {data.shape}")
                return data
            except Exception as e:
                print(f"Error loading large CSV, trying chunked reading: {str(e)}")
                # Read first 10000 rows for testing
                data = pd.read_csv(file_path, nrows=10000)
                print(f"Loaded first 10000 rows from {file_path}")
                print(f"Shape: {data.shape}")
                return data
        else:
            raise ValueError("Unsupported file format. Use JSON or CSV.")
    except FileNotFoundError:
        print(f"Error: File {file_path} not found.")
        return None
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return None

def load_data_from_influxdb():
    """Load data from InfluxDB using credentials"""
    if not INFLUXDB_AVAILABLE:
        print("InfluxDB client not available. Install with: pip install influxdb-client")
        return None
    
    if not os.path.exists(INFLUXDB_CREDENTIALS_PATH):
        print(f"InfluxDB credentials file not found: {INFLUXDB_CREDENTIALS_PATH}")
        return None
    
    try:
        # Load credentials
        with open(INFLUXDB_CREDENTIALS_PATH, 'r') as stream:
            credentials = yaml.safe_load(stream)['influxdb']
        
        print(f"Connecting to InfluxDB at: {credentials['url']}")
        
        # Create client
        client = InfluxDBClient(
            url=credentials['url'], 
            token=credentials['token'], 
            org=credentials['org'], 
            timeout=60000
        )
        
        # Define query for AP data
        query = '''from(bucket: "aruba")  
            |> range(start: -24h, stop: now())
            |> filter(fn: (r) => r["_measurement"] == "adjacent-ap")
            |> filter(fn: (r) => r["ap_type"] == "valid")
            |> filter(fn: (r) => r["_field"] == "building" or
                                 r["_field"] == "floor" or
                                 r["_field"] == "monitoring-ap" or
                                 r["_field"] == "curr-rssi" or
                                 r["_field"] == "essid")
            |> group()'''
        
        print("Executing InfluxDB query...")
        query_api = client.query_api()
        data_frame = query_api.query_data_frame(org=credentials['org'], query=query)
        
        print(f"Retrieved {len(data_frame)} records from InfluxDB")
        return data_frame
        
    except Exception as e:
        print(f"Error connecting to InfluxDB: {str(e)}")
        return None

# Load data based on configuration
if USE_INFLUXDB:
    raw_data = load_data_from_influxdb()
else:
    # Try to load the RSSI CSV file first
    print("Attempting to load RSSI data...")
    raw_data = load_data_from_file(DATA_SOURCE_CSV)
    
    if raw_data is None:
        print("Primary file not available, trying alternative files...")
        for name, path in ALTERNATIVE_DATA_FILES.items():
            if os.path.exists(path):
                print(f"Trying {name}: {path}")
                raw_data = load_data_from_file(path)
                if raw_data is not None:
                    DATA_SOURCE_CSV = path  # Update the source path
                    break

if raw_data is not None:
    print(f"\nData loaded successfully. Type: {type(raw_data)}")
    if isinstance(raw_data, pd.DataFrame):
        print(f"DataFrame shape: {raw_data.shape}")
        print(f"Columns: {list(raw_data.columns)}")
    elif isinstance(raw_data, list):
        print(f"Number of records: {len(raw_data)}")
    elif isinstance(raw_data, dict):
        print(f"Data keys: {list(raw_data.keys())}")
else:
    print("Failed to load data. Please check the file paths and formats.")

## 3. Convert Data to DataFrame

Convert the loaded data into a pandas DataFrame for processing.

In [ ]:
def convert_to_dataframe(data):
    """Convert raw data to pandas DataFrame"""
    if data is None:
        return None
    
    try:
        if isinstance(data, list):
            # Assuming data is a list of dictionaries
            df = pd.DataFrame(data)
        elif isinstance(data, dict):
            # Handle different JSON structures
            if 'data' in data:
                df = pd.DataFrame(data['data'])
            elif 'results' in data:
                df = pd.DataFrame(data['results'])
            else:
                # Try to flatten the dictionary
                df = pd.json_normalize(data)
        else:
            # Data is already a DataFrame
            df = data
        
        print(f"DataFrame created successfully!")
        print(f"Shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
        return df
        
    except Exception as e:
        print(f"Error converting data to DataFrame: {str(e)}")
        return None

# Convert raw data to DataFrame
df = convert_to_dataframe(raw_data)

if df is not None:
    print(f"\nDataFrame info:")
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print(f"\nFirst few rows:")
    display(df.head())
else:
    print("Failed to create DataFrame from loaded data.")

## 4. Filter and Clean Data

Filter data to include only 2.4GHz band, remove NaN values, and keep only APs from floors 2 and 3.

In [ ]:
def filter_and_clean_data(df):
    """Filter and clean the WiFi data"""
    if df is None:
        return None
    
    try:
        print(f"Original data shape: {df.shape}")
        
        # Assuming common column names - adjust based on actual data structure
        # These are typical column names for WiFi data
        possible_freq_cols = ['frequency', 'freq', 'band', 'channel']
        possible_rssi_cols = ['rssi', 'signal_strength', 'power']
        possible_bssid_cols = ['bssid', 'mac', 'ap_mac']
        possible_ap_cols = ['ap_name', 'ap', 'access_point']
        
        freq_col = None
        rssi_col = None
        bssid_col = None
        ap_col = None
        
        # Find the actual column names in the DataFrame
        for col in df.columns:
            col_lower = col.lower()
            if any(freq in col_lower for freq in possible_freq_cols):
                freq_col = col
            if any(rssi in col_lower for rssi in possible_rssi_cols):
                rssi_col = col
            if any(bssid in col_lower for bssid in possible_bssid_cols):
                bssid_col = col
            if any(ap in col_lower for ap in possible_ap_cols):
                ap_col = col
        
        print(f"Detected columns:")
        print(f"  Frequency: {freq_col}")
        print(f"  RSSI: {rssi_col}")
        print(f"  BSSID: {bssid_col}")
        print(f"  AP Name: {ap_col}")
        
        # Filter for 2.4GHz band (typically 2400-2500 MHz)
        if freq_col:
            df_filtered = df[df[freq_col].between(2400, 2500)].copy()
            print(f"After 2.4GHz filtering: {df_filtered.shape}")
        else:
            print("No frequency column found, skipping frequency filtering")
            df_filtered = df.copy()
        
        # Remove rows with NaN RSSI values
        if rssi_col:
            df_filtered = df_filtered.dropna(subset=[rssi_col])
            print(f"After removing NaN RSSI: {df_filtered.shape}")
        
        # Filter for floors 2 and 3 (look for patterns in AP names or BSSID)
        if ap_col:
            # Filter for APs containing 2F or 3F (floor indicators)
            floor_pattern = df_filtered[ap_col].str.contains('2F|3F', case=False, na=False)
            df_filtered = df_filtered[floor_pattern]
            print(f"After floor filtering (2F/3F): {df_filtered.shape}")
        elif bssid_col:
            # Alternative: filter based on BSSID patterns if available
            print("No AP name column found, using all data")
        
        print(f"Final filtered data shape: {df_filtered.shape}")
        return df_filtered
        
    except Exception as e:
        print(f"Error filtering data: {str(e)}")
        return df

# Apply filtering and cleaning
if df is not None:
    df_filtered = filter_and_clean_data(df)
    
    if df_filtered is not None and not df_filtered.empty:
        print(f"\nFiltered data summary:")
        print(f"Shape: {df_filtered.shape}")
        print(f"\nSample data:")
        display(df_filtered.head())
    else:
        print("No data remaining after filtering, or filtering failed.")
        df_filtered = df  # Use original data if filtering fails
else:
    print("No DataFrame available for filtering.")

## 5. Process BSSID and Pivot Data

Match BSSID information with known APs and pivot data to wide format with RSSI columns for each AP.

In [ ]:
def process_bssid_and_pivot(df):
    """Process BSSID information and pivot data to wide format"""
    if df is None or df.empty:
        return None
    
    try:
        # Find relevant columns
        time_cols = [col for col in df.columns if 'time' in col.lower()]
        bssid_cols = [col for col in df.columns if any(x in col.lower() for x in ['bssid', 'mac', 'ap_mac'])]
        rssi_cols = [col for col in df.columns if any(x in col.lower() for x in ['rssi', 'signal', 'power'])]
        ap_cols = [col for col in df.columns if any(x in col.lower() for x in ['ap_name', 'ap', 'access_point'])]
        essid_cols = [col for col in df.columns if any(x in col.lower() for x in ['essid', 'ssid', 'network'])]
        
        print(f"Available columns for pivoting:")
        print(f"  Time columns: {time_cols}")
        print(f"  BSSID columns: {bssid_cols}")
        print(f"  RSSI columns: {rssi_cols}")
        print(f"  AP name columns: {ap_cols}")
        print(f"  ESSID columns: {essid_cols}")
        
        # Use the first available column of each type
        time_col = time_cols[0] if time_cols else None
        bssid_col = bssid_cols[0] if bssid_cols else None
        rssi_col = rssi_cols[0] if rssi_cols else None
        ap_col = ap_cols[0] if ap_cols else None
        essid_col = essid_cols[0] if essid_cols else None
        
        if not (rssi_col and (bssid_col or ap_col)):
            print("Missing required columns for pivoting (need RSSI and BSSID/AP name)")
            return df
        
        # Create AP identifier (prefer AP name over BSSID)
        if ap_col:
            df['ap_identifier'] = df[ap_col]
        else:
            df['ap_identifier'] = df[bssid_col]
        
        # Create unique AP names for RSSI columns
        df['rssi_column'] = 'rssi_' + df['ap_identifier'].astype(str)
        
        # Pivot the data
        pivot_cols = []
        if time_col:
            pivot_cols.append(time_col)
        if essid_col:
            pivot_cols.append(essid_col)
        if bssid_col and ap_col:  # Keep original BSSID if we have both
            pivot_cols.append(bssid_col)
        
        # Add any other non-pivoting columns
        other_cols = [col for col in df.columns 
                     if col not in [rssi_col, ap_col if ap_col else bssid_col, 'ap_identifier', 'rssi_column']
                     and col not in pivot_cols]
        pivot_cols.extend(other_cols)
        
        if pivot_cols:
            # Group by the pivot columns and create RSSI columns for each AP
            df_pivot = df.pivot_table(
                index=pivot_cols,
                columns='rssi_column',
                values=rssi_col,
                aggfunc='mean'  # Average if multiple readings per time/AP
            ).reset_index()
            
            # Flatten column names
            df_pivot.columns.name = None
            
            print(f"Pivoted data shape: {df_pivot.shape}")
            print(f"Number of RSSI columns created: {len([col for col in df_pivot.columns if col.startswith('rssi_')])}")
            
            return df_pivot
        else:
            print("No suitable columns found for pivoting")
            return df
            
    except Exception as e:
        print(f"Error in pivoting data: {str(e)}")
        print("Returning original data")
        return df

# Process BSSID and pivot data
if df_filtered is not None:
    df_pivoted = process_bssid_and_pivot(df_filtered)
    
    if df_pivoted is not None:
        print(f"\nPivoted data info:")
        print(f"Shape: {df_pivoted.shape}")
        rssi_columns = [col for col in df_pivoted.columns if col.startswith('rssi_')]
        print(f"RSSI columns: {len(rssi_columns)}")
        if rssi_columns:
            print(f"Example RSSI columns: {rssi_columns[:5]}")
        
        print(f"\nSample pivoted data:")
        display(df_pivoted.head())
    else:
        print("Pivoting failed, using filtered data")
        df_pivoted = df_filtered
else:
    print("No filtered data available for pivoting")

## 6. Load AP Coordinate Data

Load AP location information from CSV files containing AP names and their X, Y, Z coordinates.

In [ ]:
def create_sample_ap_coordinates_from_bss_table():
    """Create sample AP coordinates from the BSS table if coordinate files don't exist"""
    try:
        # Load the AP BSS table to get AP names
        if os.path.exists(DATA_SOURCE_JSON):
            with open(DATA_SOURCE_JSON, 'r') as f:
                bss_data = json.load(f)
            
            if "Aruba AP BSS Table" in bss_data:
                ap_list = bss_data["Aruba AP BSS Table"]
                
                # Extract unique AP names for floors 2 and 3
                floor_2_aps = []
                floor_3_aps = []
                
                for ap in ap_list:
                    ap_name = ap.get("ap name", "")
                    if "_2F_" in ap_name:
                        floor_2_aps.append(ap_name)
                    elif "_3F_" in ap_name:
                        floor_3_aps.append(ap_name)
                
                # Remove duplicates
                floor_2_aps = list(set(floor_2_aps))
                floor_3_aps = list(set(floor_3_aps))
                
                print(f"Found {len(floor_2_aps)} unique APs for floor 2")
                print(f"Found {len(floor_3_aps)} unique APs for floor 3")
                
                # Create sample coordinate files
                coordinates = {}
                
                if floor_2_aps:
                    # Create sample coordinates for floor 2 (Z=6 meters)
                    coords_2f = []
                    for i, ap_name in enumerate(floor_2_aps):
                        # Create a grid layout for demonstration
                        x = (i % 10) * 10  # 10 meter spacing
                        y = (i // 10) * 10
                        z = 6.0  # 2nd floor height
                        coords_2f.append([ap_name, x, y, z])
                    
                    df_2f = pd.DataFrame(coords_2f, columns=['AP_Name', 'X', 'Y', 'Z'])
                    df_2f.to_csv('2f.csv', index=False)
                    coordinates['2f'] = df_2f
                    print(f"Created 2f.csv with {len(df_2f)} APs")
                
                if floor_3_aps:
                    # Create sample coordinates for floor 3 (Z=9 meters)
                    coords_3f = []
                    for i, ap_name in enumerate(floor_3_aps):
                        # Create a grid layout for demonstration
                        x = (i % 10) * 10  # 10 meter spacing
                        y = (i // 10) * 10
                        z = 9.0  # 3rd floor height
                        coords_3f.append([ap_name, x, y, z])
                    
                    df_3f = pd.DataFrame(coords_3f, columns=['AP_Name', 'X', 'Y', 'Z'])
                    df_3f.to_csv('3f.csv', index=False)
                    coordinates['3f'] = df_3f
                    print(f"Created 3f.csv with {len(df_3f)} APs")
                
                return coordinates
                
    except Exception as e:
        print(f"Error creating coordinates from BSS table: {str(e)}")
        return {}

def load_ap_coordinates():
    """Load AP coordinate data from CSV files"""
    ap_coords = {}
    
    # First try to load existing coordinate files
    for floor, filename in AP_COORD_FILES.items():
        try:
            if os.path.exists(filename):
                # Assuming CSV format: AP_Name,X_Coordinate,Y_Coordinate,Z_Coordinate
                df_coords = pd.read_csv(filename)
                print(f"Loaded existing {filename}: {df_coords.shape}")
                
                # Standardize column names
                columns = df_coords.columns.tolist()
                if len(columns) >= 4:
                    df_coords.columns = ['AP_Name', 'X', 'Y', 'Z']
                elif len(columns) == 3:
                    df_coords.columns = ['AP_Name', 'X', 'Y']
                    df_coords['Z'] = 0  # Default Z coordinate
                
                # Store coordinates for this floor
                ap_coords[floor] = df_coords
                print(f"  Floor {floor}: {len(df_coords)} APs")
                print(f"  Sample: {df_coords.head(2).to_string()}")
                
            else:
                print(f"File {filename} not found for floor {floor}")
                
        except Exception as e:
            print(f"Error loading {filename}: {str(e)}")
    
    # If no coordinate files found, create sample ones
    if not ap_coords:
        print("No existing coordinate files found. Creating sample coordinates from BSS table...")
        ap_coords = create_sample_ap_coordinates_from_bss_table()
    
    # Combine all floor data
    if ap_coords:
        all_coords = pd.concat(ap_coords.values(), ignore_index=True)
        print(f"\nCombined AP coordinates: {all_coords.shape}")
        return all_coords
    else:
        print("No AP coordinate files loaded successfully and couldn't create sample files")
        return None

# Load AP coordinate data
ap_coordinates = load_ap_coordinates()

if ap_coordinates is not None:
    print(f"\nAP Coordinates Summary:")
    print(f"Total APs: {len(ap_coordinates)}")
    print(f"Columns: {list(ap_coordinates.columns)}")
    print(f"\nSample coordinate data:")
    display(ap_coordinates.head())
    
    # Show coordinate ranges
    if 'X' in ap_coordinates.columns and 'Y' in ap_coordinates.columns:
        print(f"\nCoordinate ranges:")
        print(f"X: {ap_coordinates['X'].min():.2f} to {ap_coordinates['X'].max():.2f}")
        print(f"Y: {ap_coordinates['Y'].min():.2f} to {ap_coordinates['Y'].max():.2f}")
        if 'Z' in ap_coordinates.columns:
            print(f"Z: {ap_coordinates['Z'].min():.2f} to {ap_coordinates['Z'].max():.2f}")
else:
    print("No AP coordinate data available")

## 6.1 Custom AP Position Layout

Create custom AP positions to match your desired floor layout as shown in the scatter plots.

In [ ]:
def create_custom_ap_positions():
    """Create custom AP positions to match the desired layout from scatter plots"""
    
    # Custom positions for Floor 2F (Z = 6.0)
    # Based on the left scatter plot showing APs distributed across the floor
    floor_2f_positions = [
        # Left side APs
        ("D1_2F_AP001", 90, 175, 6.0),
        ("D1_2F_AP002", 100, 145, 6.0),
        ("D1_2F_AP003", 95, 115, 6.0),
        ("D1_2F_AP004", 90, 90, 6.0),
        
        # Center-left APs
        ("D1_2F_AP005", 130, 145, 6.0),
        ("D1_2F_AP006", 125, 115, 6.0),
        ("D1_2F_AP007", 135, 90, 6.0),
        
        # Center APs
        ("D1_2F_AP008", 320, 225, 6.0),
        ("D1_2F_AP009", 340, 225, 6.0),
        ("D1_2F_AP010", 360, 225, 6.0),
        
        # Right side APs
        ("D1_2F_AP011", 480, 145, 6.0),
        ("D1_2F_AP012", 490, 115, 6.0),
        ("D1_2F_AP013", 500, 90, 6.0),
        ("D1_2F_AP014", 520, 65, 6.0),
        ("D1_2F_AP015", 500, 35, 6.0),
        
        # Top section
        ("D1_2F_AP016", 110, 260, 6.0),
        ("D1_2F_AP017", 130, 260, 6.0),
        ("D1_2F_AP018", 520, 175, 6.0),
        ("D1_2F_AP019", 540, 195, 6.0),
        ("D1_2F_AP020", 520, 225, 6.0)
    ]
    
    # Custom positions for Floor 3F (Z = 9.0)
    # Based on the right scatter plot showing more APs in different pattern
    floor_3f_positions = [
        # Left cluster
        ("D1_3F_AP001", 80, 195, 9.0),
        ("D1_3F_AP002", 95, 170, 9.0),
        ("D1_3F_AP003", 85, 140, 9.0),
        ("D1_3F_AP004", 90, 110, 9.0),
        ("D1_3F_AP005", 85, 85, 9.0),
        
        # Center-left column
        ("D1_3F_AP006", 150, 140, 9.0),
        ("D1_3F_AP007", 155, 110, 9.0),
        ("D1_3F_AP008", 150, 85, 9.0),
        
        # Center area
        ("D1_3F_AP009", 190, 215, 9.0),
        ("D1_3F_AP010", 220, 215, 9.0),
        ("D1_3F_AP011", 250, 215, 9.0),
        ("D1_3F_AP012", 280, 215, 9.0),
        ("D1_3F_AP013", 270, 270, 9.0),
        ("D1_3F_AP014", 300, 270, 9.0),
        
        # Right side clusters
        ("D1_3F_AP015", 360, 215, 9.0),
        ("D1_3F_AP016", 390, 215, 9.0),
        ("D1_3F_AP017", 420, 215, 9.0),
        ("D1_3F_AP018", 450, 215, 9.0),
        ("D1_3F_AP019", 480, 215, 9.0),
        ("D1_3F_AP020", 510, 215, 9.0),
        
        # Bottom right
        ("D1_3F_AP021", 380, 170, 9.0),
        ("D1_3F_AP022", 410, 170, 9.0),
        ("D1_3F_AP023", 440, 140, 9.0),
        ("D1_3F_AP024", 470, 140, 9.0),
        ("D1_3F_AP025", 500, 170, 9.0),
        
        # Additional scattered APs
        ("D1_3F_AP026", 320, 140, 9.0),
        ("D1_3F_AP027", 350, 140, 9.0),
        ("D1_3F_AP028", 400, 110, 9.0),
        ("D1_3F_AP029", 430, 110, 9.0),
        ("D1_3F_AP030", 460, 85, 9.0),
        ("D1_3F_AP031", 490, 85, 9.0),
        ("D1_3F_AP032", 520, 85, 9.0),
        
        # Top area
        ("D1_3F_AP033", 200, 270, 9.0),
        ("D1_3F_AP034", 230, 270, 9.0),
        ("D1_3F_AP035", 350, 270, 9.0),
        ("D1_3F_AP036", 380, 270, 9.0),
        ("D1_3F_AP037", 410, 270, 9.0),
        ("D1_3F_AP038", 440, 270, 9.0),
        ("D1_3F_AP039", 470, 270, 9.0),
        ("D1_3F_AP040", 500, 270, 9.0),
        
        # Bottom scattered
        ("D1_3F_AP041", 320, 55, 9.0),
        ("D1_3F_AP042", 350, 55, 9.0),
        ("D1_3F_AP043", 380, 30, 9.0),
        ("D1_3F_AP044", 410, 30, 9.0),
        ("D1_3F_AP045", 440, 55, 9.0),
        ("D1_3F_AP046", 470, 55, 9.0),
        ("D1_3F_AP047", 500, 30, 9.0),
        ("D1_3F_AP048", 520, 55, 9.0)
    ]
    
    # Create DataFrames
    df_2f = pd.DataFrame(floor_2f_positions, columns=['AP_Name', 'X', 'Y', 'Z'])
    df_3f = pd.DataFrame(floor_3f_positions, columns=['AP_Name', 'X', 'Y', 'Z'])
    
    return df_2f, df_3f

def save_custom_ap_positions():
    """Save the custom AP positions to CSV files"""
    try:
        df_2f, df_3f = create_custom_ap_positions()
        
        # Save to CSV files
        df_2f.to_csv('2f.csv', index=False)
        df_3f.to_csv('3f.csv', index=False)
        
        print(f"✅ Created custom AP position files:")
        print(f"  2f.csv: {len(df_2f)} APs")
        print(f"  3f.csv: {len(df_3f)} APs")
        
        # Show coordinate ranges
        print(f"\nFloor 2F coordinate ranges:")
        print(f"  X: {df_2f['X'].min():.0f} to {df_2f['X'].max():.0f}")
        print(f"  Y: {df_2f['Y'].min():.0f} to {df_2f['Y'].max():.0f}")
        print(f"  Z: {df_2f['Z'].unique()}")
        
        print(f"\nFloor 3F coordinate ranges:")
        print(f"  X: {df_3f['X'].min():.0f} to {df_3f['X'].max():.0f}")
        print(f"  Y: {df_3f['Y'].min():.0f} to {df_3f['Y'].max():.0f}")
        print(f"  Z: {df_3f['Z'].unique()}")
        
        return df_2f, df_3f
        
    except Exception as e:
        print(f"❌ Error creating custom positions: {str(e)}")
        return None, None

# Create and save custom AP positions
print("Creating custom AP positions to match your scatter plot layout...")
df_2f_custom, df_3f_custom = save_custom_ap_positions()

if df_2f_custom is not None and df_3f_custom is not None:
    print(f"\n📍 AP Position Summary:")
    print(f"Floor 2F: {len(df_2f_custom)} APs positioned")
    print(f"Floor 3F: {len(df_3f_custom)} APs positioned")
    
    # Display sample positions
    print(f"\nSample Floor 2F positions:")
    display(df_2f_custom.head())
    
    print(f"\nSample Floor 3F positions:")
    display(df_3f_custom.head())
else:
    print("Failed to create custom positions")

In [ ]:
import matplotlib.pyplot as plt

def visualize_custom_ap_layout(df_2f, df_3f):
    """Visualize the custom AP layout to match your scatter plots"""
    
    # Create subplot for both floors
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot Floor 2F
    ax1.scatter(df_2f['X'], df_2f['Y'], c='blue', s=50, alpha=0.7)
    ax1.set_title('Scatter Plot of Positions (2F)')
    ax1.set_xlabel('X Position')
    ax1.set_ylabel('Y Position')
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(50, 550)
    ax1.set_ylim(25, 275)
    
    # Plot Floor 3F
    ax2.scatter(df_3f['X'], df_3f['Y'], c='blue', s=50, alpha=0.7)
    ax2.set_title('Scatter Plot of Positions (3F)')
    ax2.set_xlabel('X Position')
    ax2.set_ylabel('Y Position')
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(50, 550)
    ax2.set_ylim(25, 275)
    
    plt.tight_layout()
    plt.show()
    
    print(f"📊 Custom AP Layout Visualization:")
    print(f"Floor 2F: {len(df_2f)} APs distributed")
    print(f"Floor 3F: {len(df_3f)} APs distributed")

# Visualize the custom layout if positions were created successfully
if df_2f_custom is not None and df_3f_custom is not None:
    visualize_custom_ap_layout(df_2f_custom, df_3f_custom)

## 📝 How to Modify AP Positions

To customize your AP positions to match your desired layout:

### 1. **Edit the Position Coordinates**
In the `create_custom_ap_positions()` function above, modify the coordinates:
```python
# Format: (AP_Name, X_coordinate, Y_coordinate, Z_coordinate)
("D1_2F_AP001", 90, 175, 6.0),  # X=90, Y=175, Floor 2 (Z=6.0)
("D1_3F_AP001", 80, 195, 9.0),  # X=80, Y=195, Floor 3 (Z=9.0)
```

### 2. **Coordinate System**
- **X-axis**: 0-550 (building width)
- **Y-axis**: 0-275 (building length) 
- **Z-axis**: 6.0 for Floor 2F, 9.0 for Floor 3F

### 3. **Add/Remove APs**
- Add more APs by adding tuples to the lists
- Remove APs by deleting tuples
- Change AP names to match your actual AP naming convention

### 4. **Apply Changes**
After modifying coordinates:
1. Run the `save_custom_ap_positions()` function
2. This creates new `2f.csv` and `3f.csv` files
3. Use these files in your main pipeline

### 5. **Use in Main Pipeline**
The generated CSV files will be automatically used by your `run_all.bash` script when it processes the data.

## 7. Merge AP Location with Signal Data

Merge AP coordinate information with the signal strength DataFrame.

In [ ]:
def merge_ap_locations(df_signal, df_coords):
    """Merge AP location data with signal strength data"""
    if df_signal is None or df_coords is None:
        print("Cannot merge: missing signal data or coordinate data")
        return df_signal
    
    try:
        # Get RSSI columns that correspond to APs
        rssi_columns = [col for col in df_signal.columns if col.startswith('rssi_')]
        
        if not rssi_columns:
            print("No RSSI columns found for merging")
            return df_signal
        
        # Extract AP names from RSSI column names
        ap_names_in_signal = [col.replace('rssi_', '') for col in rssi_columns]
        print(f"APs in signal data: {len(ap_names_in_signal)}")
        print(f"Sample AP names: {ap_names_in_signal[:5]}")
        
        # Create a mapping between AP names in signal data and coordinate data
        coord_ap_names = df_coords['AP_Name'].tolist()
        print(f"APs in coordinate data: {len(coord_ap_names)}")
        print(f"Sample coordinate AP names: {coord_ap_names[:5]}")
        
        # Find matching APs
        matched_aps = []
        ap_coordinates_map = {}
        
        for signal_ap in ap_names_in_signal:
            # Try exact match first
            if signal_ap in coord_ap_names:
                matched_aps.append(signal_ap)
                coord_row = df_coords[df_coords['AP_Name'] == signal_ap].iloc[0]
                ap_coordinates_map[signal_ap] = {
                    'X': coord_row['X'],
                    'Y': coord_row['Y'],
                    'Z': coord_row.get('Z', 0)
                }
            else:
                # Try partial matching (e.g., case insensitive or substring)
                for coord_ap in coord_ap_names:
                    if signal_ap.lower() in coord_ap.lower() or coord_ap.lower() in signal_ap.lower():
                        matched_aps.append(signal_ap)
                        coord_row = df_coords[df_coords['AP_Name'] == coord_ap].iloc[0]
                        ap_coordinates_map[signal_ap] = {
                            'X': coord_row['X'],
                            'Y': coord_row['Y'],
                            'Z': coord_row.get('Z', 0)
                        }
                        break
        
        print(f\"Matched APs: {len(matched_aps)}\")\n        print(f\"Match rate: {len(matched_aps)/len(ap_names_in_signal)*100:.1f}%\")\n        \n        if matched_aps:\n            # Create coordinate columns for matched APs\n            df_merged = df_signal.copy()\n            \n            # Add coordinate information for each matched AP\n            for ap_name in matched_aps:\n                rssi_col = f'rssi_{ap_name}'\n                if rssi_col in df_merged.columns:\n                    coords = ap_coordinates_map[ap_name]\n                    df_merged[f'{rssi_col}_X'] = coords['X']\n                    df_merged[f'{rssi_col}_Y'] = coords['Y']\n                    df_merged[f'{rssi_col}_Z'] = coords['Z']\n            \n            print(f\"Added coordinate columns for {len(matched_aps)} APs\")\n            print(f\"Final merged data shape: {df_merged.shape}\")\n            return df_merged\n        else:\n            print(\"No matching APs found between signal and coordinate data\")\n            return df_signal\n            \n    except Exception as e:\n        print(f\"Error merging AP locations: {str(e)}\")\n        return df_signal\n\n# Merge AP locations with signal data\nif df_pivoted is not None and ap_coordinates is not None:\n    df_final = merge_ap_locations(df_pivoted, ap_coordinates)\n    \n    if df_final is not None:\n        print(f\"\\nFinal merged dataset:\")\n        print(f\"Shape: {df_final.shape}\")\n        \n        # Show column types\n        rssi_cols = [col for col in df_final.columns if col.startswith('rssi_') and not col.endswith(('_X', '_Y', '_Z'))]\n        coord_cols = [col for col in df_final.columns if col.endswith(('_X', '_Y', '_Z'))]\n        other_cols = [col for col in df_final.columns if col not in rssi_cols and col not in coord_cols]\n        \n        print(f\"RSSI columns: {len(rssi_cols)}\")\n        print(f\"Coordinate columns: {len(coord_cols)}\")\n        print(f\"Other columns: {len(other_cols)}\")\n        \n        print(f\"\\nSample merged data:\")\n        display(df_final.head())\n    else:\n        print(\"Merging failed\")\n        df_final = df_pivoted\nelse:\n    print(\"Cannot merge: missing pivoted data or coordinate data\")\n    df_final = df_pivoted if df_pivoted is not None else df_filtered

## 8. Export Processed Data to CSV

Save the final processed and merged dataset to CSV file for further analysis.

In [ ]:
def export_to_csv(df, filename):
    """Export DataFrame to CSV file"""
    if df is None or df.empty:
        print("No data to export")
        return False
    
    try:
        # Create output directory if it doesn't exist
        output_dir = os.path.dirname(filename) if os.path.dirname(filename) else '.'
        os.makedirs(output_dir, exist_ok=True)
        
        # Export to CSV
        df.to_csv(filename, index=False)
        
        print(f"Data exported successfully to: {filename}")
        print(f"File size: {os.path.getsize(filename) / 1024:.2f} KB")
        print(f"Shape: {df.shape}")
        
        # Show summary statistics
        rssi_cols = [col for col in df.columns if col.startswith('rssi_') and not col.endswith(('_X', '_Y', '_Z'))]
        if rssi_cols:
            print(f"\\nRSSI Statistics:")
            rssi_data = df[rssi_cols]
            print(f"  Non-null RSSI values: {rssi_data.count().sum()}\")\n            print(f\"  Average RSSI range: {rssi_data.min().min():.1f} to {rssi_data.max().max():.1f} dBm\")\n            print(f\"  Coverage: {(rssi_data.count() / len(df) * 100).mean():.1f}% average\")\n        \n        return True\n        \n    except Exception as e:\n        print(f\"Error exporting data: {str(e)}\")\n        return False\n\n# Export the final processed data\nif df_final is not None:\n    success = export_to_csv(df_final, OUTPUT_FILE)\n    \n    if success:\n        print(f\"\\n✅ Data acquisition process completed successfully!\")\n        print(f\"\\nFinal dataset summary:\")\n        print(f\"  Output file: {OUTPUT_FILE}\")\n        print(f\"  Total records: {len(df_final)}\")\n        print(f\"  Total columns: {len(df_final.columns)}\")\n        \n        # Categorize columns\n        rssi_cols = [col for col in df_final.columns if col.startswith('rssi_') and not col.endswith(('_X', '_Y', '_Z'))]\n        coord_cols = [col for col in df_final.columns if col.endswith(('_X', '_Y', '_Z'))]\n        other_cols = [col for col in df_final.columns if col not in rssi_cols and col not in coord_cols]\n        \n        print(f\"  RSSI measurements: {len(rssi_cols)} APs\")\n        print(f\"  Coordinate data: {len(coord_cols)//3 if coord_cols else 0} APs with locations\")\n        print(f\"  Metadata columns: {len(other_cols)}\")\n        \n        print(f\"\\nColumn breakdown:\")\n        if other_cols:\n            print(f\"  Metadata: {other_cols}\")\n        if rssi_cols:\n            print(f\"  RSSI columns (first 5): {rssi_cols[:5]}\")\n        if coord_cols:\n            print(f\"  Coordinate columns (first 6): {coord_cols[:6]}\")\n    else:\n        print(\"❌ Export failed\")\nelse:\n    print(\"❌ No final data available for export\")\n\nprint(f\"\\n=== Data Acquisition Process Complete ===\")"